In [10]:
import matplotlib
matplotlib.use('Agg')  # noqa
import matplotlib.pyplot as plt
import numpy as np

In [11]:
def forward_euler(y0, delta_t, dydt):
    y_new = y0 + delta_t * dydt
    return y_new

In [12]:
def generate_t_list(t0, dt, tmax):
    t_list = []
    # If 0<dt<1, can't use range function
    if (dt > 0 and dt < 1):
        count = t0
        while (count < tmax):
            t_list.append(count)
            count += dt
    else:  # Get list of times using range
        t_list = list(range(t0, tmax, dt))
    return t_list

In [13]:
def initialize_array(values):
    arr = []
    for x in values:
        temp = [x]
        arr.append(temp)
    return arr

In [14]:
def calc_weighted_sum(v1, v2):
    wsum = 0
    for i,val in enumerate(v1):
        wsum += (val*v2[i])
    return wsum

In [136]:
def check_size_thresh(value):
    if (abs(value) < 10**(-10)):
        return 0
    else:
        return value

In [15]:
def apply_vector_euler(curr_X_v, dt, dX_v):
    updated_X = []
    for i, Xi in enumerate(curr_X_v):  # For each compartment, apply forward Euler
        Xnew = forward_euler(Xi, dt, dX_v[i])
        updated_X.append(Xnew)
    return updated_X

In [139]:
def simulate_malaria_ladder(SH_0_v, IH_0_v, EH_0_v, RH_0_v, SM_0, EM_0, IM_0, mu_H, mu_M, beta_H_v, beta_M, gamma_M, nu, sigma_H_v, gamma_H_v, delta_H_v, psi_H_v, dt, tmax):
    '''
    Numerical simulation of a malaria model with aging
    - Uniform birth/death
    - Compartmenta 'ladder' of humans, one kind of mosquito
    - Calculated using absolute numbers, not proportions
    
    Humans are simulated using an SEIR model
    Mosquitoes are simulated using an SI model
    Time is in units of days
    Human paramaters are parsed into the function in a list
    
    Variables:
    mu_H = Natural human death rate
    mu_M = Natural mosquito death rate
    beta_v = Human exposure rates in list/vector
    beta_M = Mosquito infection rate (mosquito biting rate * chance of infection after bite)
    nu = Tuning factor to scale probability that exposed individuals can pass infection to mosquito
    sigma_v = Rates of development of clinical symptoms in humans after exposure (vector)
    gamma_v = Human recovery rates (vector)
    delta_v = Infection induced mortality rates in humans (vector)
    psi_v = Human loss of immunity rates (return to susceptible) (vector)
    '''
    NH = sum(RH_0_v) + sum(SH_0_v) + sum(IH_0_v) + sum(EH_0_v)
    NM = SM_0 + EM_0 + IM_0
    
    # Storage of values over time
    SH_arr = initialize_array(SH_0_v)
    EH_arr = initialize_array(EH_0_v)
    IH_arr = initialize_array(IH_0_v)
    RH_arr = initialize_array(RH_0_v)
    
    SM = [SM_0]
    EM = [EM_0]
    IM = [IM_0]
    
    # Set initial values
    curr_SH_v = SH_0_v
    curr_EH_v = EH_0_v
    curr_IH_v = IH_0_v
    curr_RH_v = RH_0_v
    
    curr_SM = SM_0
    curr_EM = EM_0
    curr_IM = IM_0
    
    # If 0<dt<1, can't use range function
    t_list = generate_t_list(t0=0, dt=dt, tmax=tmax)

    # Track differentials for forward euler. Must update values SIMULTANEOUSLY, otherwise updates from compartment 1 would have an effect on comp 2 calculations, etc.
    num_comp = len(curr_SH_v)
    dSH_v = [0] * num_comp
    dEH_v = [0] * num_comp
    dIH_v = [0] * num_comp
    dRH_v = [0] * num_comp
    for t in t_list:
        # Calculate differential changes for forward euler
        # Round down numbers smaller than 10^-10 precision
        for i, curr_SH_i in enumerate(curr_SH_v):
            # For each compartment, make common calculations for E, I, R
            
            # dEH(i) = Exposure rate - infection rate - death rate
            dE_temp = (beta_H_v[i] * curr_SH_v[i] * curr_IM / NM) - ((sigma_H_v[i] + mu_H) * curr_EH_v[i])
            dEH_v[i] = check_size_thresh(dE_temp)
            
            # dIH(i) = Infection rate - recovery rate - natural death rate - disease associated mortality
            dI_temp = (sigma_H_v[i] * curr_EH_v[i]) - ((gamma_H_v[i] + mu_H + delta_H_v[i]) * curr_IH_v[i])
            dIH_v[i] = check_size_thresh(dI_temp)
            
            # dRH(i) = Recovery rate - unrecovery rate - death rate
            dR_temp = (gamma_H_v[i] * curr_IH_v[i]) - ((psi_H_v[i] + mu_H) * curr_RH_v[i])
            dRH_v[i] = check_size_thresh(dR_temp)
            
            # For S, modify calculations according to compartment number. First and last compartments have different rates
            if (i == 0):  # First compartment
                mal_death = calc_weighted_sum(delta_H_v, curr_IH_v)
                # dSH(1) = birth - exposure rate - death rate
                dS_temp = (mu_H * NH + mal_death) - (beta_H_v[i] * curr_SH_v[i] * curr_IM / NM) - (mu_H * curr_SH_v[i])
                dSH_v[i] = check_size_thresh(dS_temp)
            elif (i == (len(curr_SH_v) - 1)):  # Last compartment
                # dSH(n) = Unrecovery - exposure - death
                dS_temp = (psi_H_v[i-1] * curr_RH_v[i-1] + psi_H_v[i] * curr_RH_v[i]) - (beta_H_v[i] * curr_SH_v[i] * curr_IM / NM) - (mu_H * curr_SH_v[i])
                dSH_v[i] = check_size_thresh(dS_temp)
            else:  # Any other compartment
                # dSH(i) = Unrecovery - exposure - death
                dS_temp = (psi_H_v[i-1] * curr_RH_v[i-1]) - (beta_H_v[i] * curr_SH_v[i] * curr_IM / NM) - (mu_H * curr_SH_v[i])
                dSH_v[i] = check_size_thresh(dS_temp)

        ############################################################################
        #print(sum(dSH_v) + sum(dEH_v) + sum(dIH_v) + sum(dRH_v))
        #print(curr_RH_v)
        # dSM = Birth - exposure rate - death rate
        dSM = (mu_M * NM) - ((beta_M * (nu * sum(curr_EH_v) + sum(curr_IH_v)) / NH) * curr_SM) - (mu_M * curr_SM)

        # dEM = exposure rate - infection rate - death rate
        dEM = ((beta_M * (nu * sum(curr_EH_v) + sum(curr_IH_v)) / NH) * curr_SM) - (gamma_M * curr_EM) - (mu_M * curr_EM)

        # dIM = Infection rate - death rate
        dIM = (gamma_M * curr_EM) - (mu_M * curr_IM)

        curr_SM = forward_euler(curr_SM, dt, dSM)
        curr_EM = forward_euler(curr_EM, dt, dEM)
        curr_IM = forward_euler(curr_IM, dt, dIM)
        
        # Simultaneously update population numbers
        curr_SH_v = apply_vector_euler(curr_SH_v, dt, dSH_v)
        curr_EH_v = apply_vector_euler(curr_EH_v, dt, dEH_v)
        curr_IH_v = apply_vector_euler(curr_IH_v, dt, dIH_v)
        curr_RH_v = apply_vector_euler(curr_RH_v, dt, dRH_v)

        # Store new human population values
        for i, curr_SH_i in enumerate(curr_SH_v):
            SH_arr[i].append(curr_SH_v[i])
            EH_arr[i].append(curr_EH_v[i])
            IH_arr[i].append(curr_IH_v[i])
            RH_arr[i].append(curr_RH_v[i])

        SM.append(curr_SM)
        EM.append(curr_EM)
        IM.append(curr_IM)
    
    # Complete list of times, return results
    t_list.append(t_list[-1] + dt)

    return t_list, SH_arr, EH_arr, IH_arr, RH_arr, SM, EM, IM 

In [140]:
def generate_uniform_list(start, end, num):
    increment = (start - end) / (num - 1)
    curr = start
    output_list = [start]
    for num in range(num - 1):
        curr += increment
        output_list.append(curr)
    return output_list

In [141]:
SH_0_v = [2000000, 1000000, 750000, 750000, 500000]
EH_0_v = [0, 0, 0, 0, 0]
IH_0_v = [0, 0, 0, 0 ,0]
RH_0_v = [0, 0, 0, 0, 0]
SM_0 = 2490000
EM_0 = 0
IM_0 = 10000
mu_H = 0.00004
mu_M = 0.05
beta_H_v = generate_uniform_list(start=0.1, end=0.025, num=5)
beta_M = 0.417
gamma_M = 0.0714  # Four days (guess right now)
nu = 0.5
sigma_v = generate_uniform_list(0.1033, 0.0833, num=5)
gamma_v = generate_uniform_list(0.0014, 0.0035, num=5)
delta_v = generate_uniform_list(0.0003454, 0.0000174, num=5)
psi_v = generate_uniform_list(0.0033, 0.0027, num=5)
dt = 1  # Time in days
tmax = 18250

t_list, SH_arr, EH_arr, IH_arr, RH_arr, SM, EM, IM = simulate_malaria_ladder(SH_0_v, IH_0_v, EH_0_v, RH_0_v, SM_0, EM_0, IM_0, mu_H, mu_M, beta_H_v, beta_M, gamma_M, nu, sigma_v, gamma_v, delta_v, psi_v, dt, tmax)

In [142]:
t_list_years = [x / 365 for x in t_list]

In [153]:
# Iteratively plot human populations
fig, ax = plt.subplots()

num_groups = len(SH_arr)
# Generate labels for each group
labels = ['SH ' + str(ndx + 1) for ndx in range(num_groups)]

# Generate an even distribution of opacities for each s line
# The last i group will be the most solid (alpha=1)
opacities = [(ndx + 1) * (0.75 / num_groups) for ndx in range(num_groups)]
opacities = list(reversed(opacities))

for ndx, SH_list in enumerate(SH_arr): # Plot S data using a loop
    ax.plot(t_list_years, SH_list, label=labels[ndx], alpha=opacities[ndx], color='orange')

ax.set_xlabel('Time (Years)')
ax.set_ylabel('Population')
ax.set_title('Human Susceptible Population vs Time')
ax.legend(loc='upper right')
ax.spines[['right','top']].set_visible(False)
plt.savefig('mml_susceptible_humans.png')
plt.close()

In [154]:
# Iteratively plot human populations
fig, ax = plt.subplots()

num_groups = len(EH_arr)
# Generate labels for each group
labels = ['EH ' + str(ndx + 1) for ndx in range(num_groups)]

# Generate an even distribution of opacities for each s line
# The last i group will be the most solid (alpha=1)
opacities = [(ndx + 1) * (0.75 / num_groups) for ndx in range(num_groups)]
opacities = list(reversed(opacities))

for ndx, EH_list in enumerate(EH_arr): # Plot S data using a loop
    ax.plot(t_list_years, EH_list, label=labels[ndx], alpha=opacities[ndx], color='green')

ax.set_xlabel('Time (Years)')
ax.set_ylabel('Population')
ax.set_title('Human Exposed Population vs Time')
ax.legend(loc='upper right')
ax.spines[['right','top']].set_visible(False)
plt.savefig('mml_exposed_humans.png')
plt.close()

In [155]:
# Iteratively plot human populations
fig, ax = plt.subplots()

num_groups = len(IH_arr)
# Generate labels for each group
labels = ['IH ' + str(ndx + 1) for ndx in range(num_groups)]

# Generate an even distribution of opacities for each s line
# The last i group will be the most solid (alpha=1)
opacities = [(ndx + 1) * (0.75 / num_groups) for ndx in range(num_groups)]
opacities = list(reversed(opacities))

for ndx, IH_list in enumerate(IH_arr): # Plot S data using a loop
    ax.plot(t_list_years, IH_list, label=labels[ndx], alpha=opacities[ndx], color='red')

ax.set_xlabel('Time (Years)')
ax.set_ylabel('Population')
ax.set_title('Human Infected Population vs Time')
ax.legend(loc='upper right')
ax.spines[['right','top']].set_visible(False)
plt.savefig('mml_infected_humans.png')
plt.close()

In [156]:
# Iteratively plot human populations
fig, ax = plt.subplots()

num_groups = len(RH_arr)
# Generate labels for each group
labels = ['RH ' + str(ndx + 1) for ndx in range(num_groups)]

# Generate an even distribution of opacities for each s line
# The last i group will be the most solid (alpha=1)
opacities = [(ndx + 1) * (0.75 / num_groups) for ndx in range(num_groups)]
opacities = list(reversed(opacities))

for ndx, RH_list in enumerate(RH_arr): # Plot S data using a loop
    ax.plot(t_list_years, RH_list, label=labels[ndx], alpha=opacities[ndx], color='blue')

ax.set_xlabel('Time (Years)')
ax.set_ylabel('Population')
ax.set_title('Human Recovered Population vs Time')
ax.legend(loc='upper right')
ax.spines[['right','top']].set_visible(False)
plt.savefig('mml_recovered_humans.png')
plt.close()

In [157]:
fig, ax = plt.subplots()
ax.plot(t_list_years,SM,label='S Mosquito',color='orange')
ax.plot(t_list_years,EM,label='E Mosquito',color='green')
ax.plot(t_list_years,IM,label='I Mosquito',color='red')
ax.set_xlabel('Time (Years)')
ax.set_ylabel('Mosquito Population')
ax.set_title('Mosquito Population vs Time')
ax.legend(loc='upper right')
ax.spines[['right','top']].set_visible(False)
plt.savefig('mml_mosquito.png')
plt.close()